In [ ]:
import sqlite3

con = sqlite3.connect('country_club.db')
f = open('country_club (2).sql','r')
s = f.read()
cur = con.cursor()
cur.execute(s)

In [ ]:
f.close()

In [17]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(
        host="localhost", user="Julia", password="admin"
    )
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Error while connecting to MySQL 2026 (HY000): SSL connection error: Failed to set ciphers to use


NameError: name 'connection' is not defined

In [19]:
import mysql.connector

mydb = mysql.connector.connect(user="root", passwd="admin", host="127.0.0.1", port = 3306, ssl_disabled = True)

cursor = mydb.cursor()

InterfaceError: 2026 (HY000): SSL connection error: Failed to set ciphers to use

In [3]:
cursor.execute("SHOW DATABASES")
for x in cursor:
    print(x)

NameError: name 'cursor' is not defined

In [ ]:
cursor.execute("CREATE DATABASE country_club")
cursor.execute("SHOW DATABASES")
for x in cursor:
    print(x)

In [ ]:
mydb = mysql.connector.connect(user="Julia", passwd="admin", database = 'country_club')
cursor = mydb.cursor()
with open('country_club.sql') as f:
    cursor.execute(f.read(), multi=True)

In [ ]:
query = ("select * from Members")
cursor.execute(query)


Welcome to the SQL mini project. For this project, you will use Springboard' online SQL platform, which you can log into through the following link:

https://sql.springboard.com/

Username: student

Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:

    i) the "Bookings" table,
    
    ii) the "Facilities" table, and
    
    iii) the "Members" table.


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [ ]:
SELECT * FROM `Facilities` WHERE `membercost` > 0

#### Q2: How many facilities do not charge a fee to members? 

In [ ]:
SELECT COUNT(*) FROM `Facilities` WHERE `membercost` = 0

#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
SELECT `facid`, `name`, `membercost`, `monthlymaintenance` FROM `Facilities` WHERE `membercost` > 0 AND `membercost` <  `monthlymaintenance` * 0.2 

#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [ ]:
SELECT * FROM `Facilities` WHERE `facid` IN (1,5)

#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.

In [ ]:
SELECT `name`, `monthlymaintenance`,  
CASE   
        WHEN  `monthlymaintenance`> 100 THEN 'expensive'
        ELSE 'cheap' 
    END
FROM `Facilities`

#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [ ]:
SELECT  `surname`, `firstname`,`joindate` 
FROM 
`Members` 
WHERE `joindate` = (SELECT MAX(`joindate`) FROM `Members`)

#### Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [ ]:
SELECT  DISTINCT CONCAT(m.surname , ' ' ,  m.firstname) as member_name, f.name as facility_name
FROM Members m
JOIN Bookings b on b.memid = m.memid
JOIN Facilities f on f.facid = b.facid
WHERE f.name LIKE 'Tennis Court %'
ORDER BY member_name

#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [ ]:
SELECT  CONCAT(m.surname , ' ' ,  m.firstname) as member_name, f.name as facility_name,
CASE   
        WHEN m.memid = 0 THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots
    END
as cost
FROM Members m
JOIN Bookings b on b.memid = m.memid
JOIN Facilities f on f.facid = b.facid
WHERE b.starttime LIKE ('2012-09-14 %')
HAVING cost > 30
ORDER BY cost DESC

#### Q9: This time, produce the same result as in Q8, but using a subquery

In [ ]:
SELECT  CONCAT(m.surname , ' ' ,  m.firstname) as member_name, subquery.facility_name, subquery.cost
FROM
(SELECT b.memid, f.name as facility_name, b.starttime,
CASE   
        WHEN b.memid = 0 THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots
    END
as cost
FROM Bookings b
JOIN Facilities f on f.facid = b.facid
) as subquery
JOIN Members m on  subquery.memid = m.memid
WHERE subquery.starttime LIKE ('2012-09-14 %')
HAVING subquery.cost > 30
ORDER BY subquery.cost DESC

#### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

In [ ]:
SELECT f.name as facility_name, sum(
CASE   
        WHEN m.memid = 0 THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots
    END
 ) as revenue
FROM Members m
JOIN Bookings b on b.memid = m.memid
JOIN Facilities f on f.facid = b.facid
GROUP BY facility_name
HAVING revenue < 1000
ORDER BY revenue
